# Initial Watch Records

## 1. Web Access

### 1. Modules & Variables

In [ ]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크 (https://github.com/ultrafunkamsterdam/undetected-chromedriver)


In [1]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [1]:
# Account Variables

info = open('secure/id_pw.txt').readlines()

account_1 = info[0].split(' ')[1].rstrip('\n')
password_1 = info[1].split(' ')[1].rstrip('\n')

account_2 = info[2].split(' ')[1].rstrip('\n')
password_2 = info[3].split(' ')[1].rstrip('\n')


### 2. Web Access

In [7]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [8]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
btn_sign_in.click()

# 계정 이메일 입력
input_account = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
input_account.send_keys(account_1)

# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
btn_next_password.click()

# 계정 비밀번호 입력
input_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
input_password.send_keys(password_1)

# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
btn_next_sign_in.click()


In [11]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지

try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


In [80]:
driver.refresh()


### 3. Data Collection

#### 1) Variables

In [52]:
# Variables (1) - Record data Tag Attributes


# 기본 영상 정보
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>
# WatchDate = container[data-date] / c-wiz 태그의 [data-date] 값

# 시청유무 판별 정보
nm_class_thumbnail = "OUPWA"  # 기록 우측에 있는 영상 정보 (썸네일) / tag = <div>
nm_class_percentage = "HmLFgd"  # 썸네일 하단 빨간 Percentage Bar / tag = <div>
# RunningTime 활용

# 시청기록 - video id
# replace(pattern, '')로 id 값 추출
id_str_pattern = "https://www.youtube.com/watch?v="


In [ ]:
# Variables (2) Google API - Youtube Data


# API 실행에 필요한 모듈
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# API 정보, API Key => API build
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
DEVELOPER_KEY = open('secure/api_key.txt').readline()

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)


#### 2) Functions

In [87]:
# Function (1) - date & day


def get_day(weekday=datetime.today().weekday()):
    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d').strftime('%Y-%m-%d')


In [82]:
# Function (2) - Category Id & Title

""" Google API - Youtube DATA API 활용

    세부 query
    - videos().list(): id 값(watch?v="여기 표시되는 값")으로 동영상 정보를 호출
        - categoryId 호출을 위해 활용
    
    - videoCategories().list(): categoryId 값(1~44)으로 카테고리 정보를 호출
        - categoryTitle 호출을 위해 활용
"""

""" 개선 필요
        categoryTitle이 영어로 호출됨
        query 옵션에서 한국어를 호출하거나, 번역해야 한다.
"""


def get_catId(input_id):
    video = youtube.videos().list(
        part='snippet',
        id=input_id
    ).execute()

    return video['items'][0]['snippet']['categoryId']


def get_catTitle(num):
    catId = youtube.videoCategories().list(
        id=num,
        part='snippet'
    ).execute()

    try:
        result = catId['items'][0]['snippet']['title']
    except:
        result = ''

    return result


In [83]:
# Function (3) - Watched_YN classifier

""" 구현 가능 Option (1)
    - 구현법
        1. 현재 레코드의 watch_time과 이전 레코드의 watch_time의 차이값 구한다
        2. 영상 길이와 비슷하면 watched, 아닐 시 not_watched

    - 문제점
        1. 마지막으로 시청한 영상은 차이를 구할 수 없다.
        2. 길이 5분 이상 광고를 방치한 경우 시간 계산에 오류가 생긴다
"""

""" 구현 가능 Option (2)
    - 구현법
        1. 시청 기록 레코드의 썸네일의 진행도 막대를 활용한다.
        2. 시청 완료하지 않은 영상에 표시되며, 태그 내부의 시청 비율 값을 추출한다.

    - 문제점
        1. 라이브 스트리밍의 경우 잠깐 보고 꺼도 진행도 막대가 출력되지 않는다.
        2. 대충 넘기기만 해도 진행도 막대가 표시된다. 
"""


def classify_watched(content):
    running_time = content.find('div', class_=nm_class_RunningTime).text
    thumbnail = content.find('div', class_=nm_class_thumbnail)
    percentage = thumbnail.find('div', class_=nm_class_percentage)

    try:
        result = int(percentage['style'].split(":")[1][:-1]) / 100

    except:
        if len(running_time) > 5: # 영상 길이가 1시간 이상(0:00:00)인 경우
            result = 0 # 미시청으로 간주
        else:
            result = 1 # 시청으로 간주

    return result


#### 2) Empty DataFrame

In [85]:
import pandas as pd
columns_rec = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category_id', 'category_title', 'watched (%)']
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%)


In [22]:
columns_ser = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time


#### 3) Web Data Collecting

In [10]:
# # 목표 갯수만큼 시청 목록(레코드) 호출

# num_target_data = 1000  # int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
# num_press_end = int(num_target_data / 100)

# for n in range(num_press_end - 1):
#     driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
#     time.sleep(2) # 전체 페이지가 뜨도록 Wait

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 200


In [53]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime):  # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            channel = content.find('div', class_=nm_class_channel).text
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            # watched_YN = input(f'{title}을 끝까지 시청하셨나요? \t')

            id = content.find('a')['href'].replace(id_str_pattern, '')
            # input(f'{title}의 Category는? \t') # 영상 분류 Model 활용을 위해
            category_id = get_catId(id)
            category_title = get_catTitle(category_id)

            df_record.loc[idx_rec] = [title, channel,
                                      running_time, watch_date, watch_time, category_id, category_title]  # watched_YN
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Search Record
1: Watch Record
2: Watch Record
3: Watch Record
4: Watch Record
5: Ad
6: Ad
7: Watch Record
8: Watch Record
9: Ad
10: Watch Record
11: Watch Record
12: Watch Record
13: Watch Record
14: Watch Record
15: Watch Record
16: Watch Record
17: Watch Record
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Watch Record
23: Watch Record
24: Watch Record
25: Watch Record
26: Watch Record
27: Watch Record
28: Watch Record
29: Watch Record
30: Watch Record
31: Watch Record
32: Watch Record
33: Watch Record
34: Watch Record
35: Watch Record
36: Watch Record
37: Watch Record
38: Watch Record
39: Watch Record
40: Watch Record
41: Watch Record
42: Watch Record
43: Watch Record
44: Watch Record
45: Watch Record
46: Watch Record
47: Watch Record
48: Watch Record
49: Community Post 	 Watch time: 오후 1:50
50: Watch Record
51: Watch Record
52: Watch Record
53: Watch Record
54: Watch Record
55: Watch Record
56: Watch Record
57: Watch Record
58: Watch Record
59: Watch Re

In [55]:
# df_record.to_csv('230212_watch_record.csv', encoding='utf-8')
df_record.head(50)


,title,channel,running_time,watch_date,watch_time,category_id,category_title
0,데이터 엔지니어는 무슨 일을 하는 사람일까? | 데이터엔지니어 직무 소개(feat....,메이크센스 MakeSense,5:39,2023-02-12,오전 12:49,26,Howto & Style
1,수상할정도로 벨코즈를 잘하는 곰,박나나,11:01,2023-02-11,오후 10:39,20,Gaming
2,마스터이 명상 무한으로 사용하는 좀비 마스터이 ㅋㅋㅋㅋ [테스터훈 뉴메타],테스터훈 마이너스,10:55,2023-02-11,오후 10:27,20,Gaming
3,"출장 수리 사기, 결국 고소했습니다",뻘짓연구소,11:13,2023-02-11,오후 9:18,28,Science & Technology
4,응애 | 젠지 vs. KT 게임1 하이라이트 | 02.11 | 2023 LCK 스프...,LCK,6:34,2023-02-11,오후 9:12,20,Gaming
5,"""𝟙𝟘𝟘"" | 젠지 vs. KT 게임3 하이라이트 | 02.11 | 2023 LCK...",LCK,10:55,2023-02-11,오후 9:01,20,Gaming
6,일 왜함? - 러스트(2),랄로,43:46,2023-02-11,오후 9:00,20,Gaming
7,요네는 돌아오는거야,PAKA,24:43,2023-02-11,오후 8:35,20,Gaming
8,럭스1위 여왕럭스 본캐 그마 솔로랭크,여왕럭스,4:53:10,2023-02-11,오후 8:14,20,Gaming
9,이 바텀 조합 살벌한데..? 페이커의 딩거 서폿! (feat.구마유시) [Faker...,T1 Faker,6:47,2023-02-11,오후 8:07,20,Gaming


In [ ]:
df_search.to_csv('230211_search_record.csv', encoding='utf-8')
df_search


In [26]:
print(
    f"입력한 시청기록 레코드는 {num_target_data}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")


입력한 시청기록 레코드는 1000개, 
실제 시청한 영상은 418개, 
광고는 404개입니다.


### Notes